### Step 1: Install Dependencies

In [8]:
!uv pip install -q langchain langchain-groq langchain-huggingface sentence-transformers faiss-cpu python-dotenv langchain-community

### Step 2: Load Environment Variables

In [9]:
import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv("GROQ_API_KEY"):
    raise ValueError("GROQ_API_KEY not found in .env file")
else:
    print("GROQ_API_KEY loaded successfully.")

GROQ_API_KEY loaded successfully.


### Step 3: Data Ingestion

In [ ]:
from langchain_community.document_loaders import TextLoader

# file path
file_path = "/home/arshad_murtaza/ars_projects/multi-doc-chat-advanced-rag-pilot_1/data/Agentic AI.txt" 

loader = TextLoader(file_path, encoding="utf8")
documents = loader.load()


print(f"Loaded {len(documents)} document(s).")
print(f"First 500 chars: {documents[0].page_content[:500]}")

Loaded 1 document(s).
First 500 chars: Understanding Agentic AI

Agentic AI refers to a new paradigm in artificial intelligence where systems are designed not just to respond to queries or perform specific tasks, but to operate autonomously towards achieving predefined goals. This involves capabilities such as planning, reasoning, executing actions, and continuously adapting to dynamic environments without constant human intervention.

Key Characteristics of Agentic AI

Agentic AI systems are distinct from traditional AI models due t


### Step 4: Text Splitting

In [11]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
text_chunks = text_splitter.split_documents(documents)

print(f"Total chunks created: {len(text_chunks)}")

Total chunks created: 21


### Step 5: Initialize HuggingFace Embeddings

In [12]:
from langchain_huggingface import HuggingFaceEmbeddings

# Using 'all-MiniLM-L6-v2' 
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

/home/arshad_murtaza/ars_projects/multi-doc-chat-advanced-rag-pilot_1/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Step 6: Create Vector Store (FAISS)

In [13]:
from langchain_community.vectorstores import FAISS

# Create the vector store
vectorstore = FAISS.from_documents(text_chunks, embeddings)


print("Vector Store created successfully.")

Vector Store created successfully.


### Step 7: Setup Retriever

In [14]:
retriever = vectorstore.as_retriever()

### Step 8: Define Prompt Template

In [15]:
from langchain_core.prompts import ChatPromptTemplate

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

prompt = ChatPromptTemplate.from_template(template)

### Step 9: Initialize Groq LLM

In [16]:
from langchain_groq import ChatGroq

# Initialize Groq Chat Model
llm_model = ChatGroq(
    model="openai/gpt-oss-20b", 
    temperature=0
)

### Step 10: Build and Run the RAG Chain

In [17]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

output_parser = StrOutputParser()

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

# Invoke the chain
response = rag_chain.invoke("tell me about Agentic AI")
print(response)

Agentic AI is a new paradigm in artificial intelligence that moves beyond simple query‑response or task‑specific behavior. Instead, these systems are designed to act autonomously, setting and pursuing their own goals within a given environment. Key characteristics include self‑directed planning, adaptive learning, and the ability to evaluate progress toward objectives. Understanding Agentic AI involves studying how such systems balance exploration and exploitation while maintaining alignment with human values. Researchers envision a future where Agentic AI can collaborate with humans, manage complex projects, and innovate independently. However, this autonomy also raises challenges in safety, accountability, and ethical governance that must be addressed as the technology matures.
